This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.2_tutorials/wallaroo-model-deploy-and-serve/demand_curve).

## Demand Curve Pipeline Tutorial

This worksheet demonstrates a Wallaroo pipeline with data preprocessing, a model, and data postprocessing.

The model is a "demand curve" that predicts the expected number of units of a product that will be sold to a customer as a function of unit price and facts about the customer. Such models can be used for price optimization or sales volume forecasting.  This is purely a "toy" demonstration, but is useful for detailing the process of working with models and pipelines.

Data preprocessing is required to create the features used by the model. Simple postprocessing prevents nonsensical estimates (e.g. negative units sold).

## Prerequisites

* An installed Wallaroo instance.
* The following Python libraries installed:
  * `os`
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.

In [1]:
import json
import wallaroo
from wallaroo.object import EntityNotFoundError
import pandas
import numpy
import conversion
from wallaroo.object import EntityNotFoundError
import pyarrow as pa

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

Now that the Wallaroo client has been initialized, we can create the workspace and call it `demandcurveworkspace`, then set it as our current workspace.  We'll also create our pipeline so it's ready when we add our models to it.

We'll set some variables and methods to create our workspace, pipelines and models.  Note that as of the July 2022 release of Wallaroo, workspace names must be unique.  Pipelines with the same name will be created as a new version when built.

In [3]:
workspace_name = 'demandcurveworkspace'
pipeline_name = 'demandcurvepipeline'
model_name = 'demandcurvemodel'
model_file_name = './models/demand_curve_v1.onnx'

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)

demandcurve_pipeline = wl.build_pipeline(pipeline_name)
demandcurve_pipeline

name,demandcurvepipeline
created,2024-04-16 19:38:36.714333+00:00
last_updated,2024-04-16 19:38:36.714333+00:00
deployed,(none)
arch,None
accel,None
tags,
versions,79b65e34-675d-4bdc-b764-1299f30fa26f
steps,
published,False


With our workspace established, we'll upload three models:


* `./models/preprocess_dc_byop.zip`: A preprocess model step that formats the data into a tensor that the model can inference from.
* `./models/demand_curve_v1.onnx`: Our demand_curve model.  We'll store the upload configuration into `demand_curve_model`.
* `./models/postprocess_dc_byop.zip`: A postprocess model step that will zero out any negative values and return the output variable as "prediction".

Note that the order we upload our models isn't important - we'll be establishing the actual process of moving data from one model to the next when we set up our pipeline.

In [5]:
demand_curve_model = wl.upload_model(model_name, 
                                     model_file_name, 
                                     framework=wallaroo.framework.Framework.ONNX
                                     ).configure(tensor_fields=["tensor"])

In [6]:
input_schema = pa.schema([
    pa.field('Date', pa.string()),
    pa.field('cust_known', pa.bool_()),
    pa.field('StockCode', pa.int32()),
    pa.field('UnitPrice', pa.float32()),
    pa.field('UnitsSold', pa.int32())
])

output_schema = pa.schema([
    pa.field('tensor', pa.list_(pa.float64()))
])

preprocess_step = wl.upload_model('curve-preprocess', 
                                  './models/preprocess_dc_byop.zip', 
                                  framework=wallaroo.framework.Framework.CUSTOM, 
                                  input_schema=input_schema, 
                                  output_schema=output_schema)

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a container runtime..
Model is attempting loading to a container runtime...........successful

Ready


In [7]:
input_schema = pa.schema([
    pa.field('variable', pa.list_(pa.float64()))
])


output_schema = pa.schema([
    pa.field('prediction', pa.list_(pa.float64()))
])

postprocess_step = wl.upload_model('curve-postprocess', 
                                   './models/postprocess_dc_byop.zip', 
                                   framework=wallaroo.framework.Framework.CUSTOM, 
                                   input_schema=input_schema, 
                                   output_schema=output_schema)

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a container runtime..
Model is attempting loading to a container runtime..........successful

Ready


With our models uploaded, we're going to create our own pipeline and give it three steps:

* The preprocess step to put the data into a tensor format.
* Then we apply the data to our `demand_curve_model`.
* And finally, we prepare our data for output with the `module_post`.

In [8]:
# now make a pipeline
demandcurve_pipeline.undeploy()
demandcurve_pipeline.clear()
demandcurve_pipeline.add_model_step(preprocess_step)
demandcurve_pipeline.add_model_step(demand_curve_model)
demandcurve_pipeline.add_model_step(postprocess_step)

name,demandcurvepipeline
created,2024-04-16 19:38:36.714333+00:00
last_updated,2024-04-16 19:38:36.714333+00:00
deployed,(none)
arch,None
accel,None
tags,
versions,79b65e34-675d-4bdc-b764-1299f30fa26f
steps,
published,False


And with that - let's deploy our model pipeline.  This usually takes about 45 seconds for the deployment to finish.

In [9]:
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("1Gi").build()
demandcurve_pipeline.deploy(deployment_config=deploy_config)

name,demandcurvepipeline
created,2024-04-16 19:38:36.714333+00:00
last_updated,2024-04-16 19:40:58.596389+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"182cdcee-ef04-416e-a4dd-964e3b3a2e9c, 79b65e34-675d-4bdc-b764-1299f30fa26f"
steps,demandcurvemodel
published,False


Everything is ready.  Let's feed our pipeline some data.  We have some information prepared with the `daily_purchasses.csv` spreadsheet.  We'll start with just one row to make sure that everything is working correctly.

In [10]:
# read in some purchase data
purchases = pandas.read_csv('daily_purchases.csv')

# start with a one-row data frame for testing
subsamp_raw = purchases.iloc[0:1,: ]
subsamp_raw

,Date,cust_known,StockCode,UnitPrice,UnitsSold
0,2010-12-01,False,21928,4.21,1


In [12]:
result = demandcurve_pipeline.infer(subsamp_raw)
display(result)

,time,in.Date,in.StockCode,in.UnitPrice,in.UnitsSold,in.cust_known,out.prediction,anomaly.count
0,2024-04-16 19:41:44.706,2010-12-01,21928,4.21,1,False,[6.680255142999893],0


We can see from the `out.prediction` field that the demand curve has a predicted slope of 6.68 from our sample data.  We can isolate that by specifying just the data output below.

In [13]:
display(result.loc[0, ['out.prediction']][0])

[6.680255142999893]

# Bulk Inference

The initial test went perfectly.  Now let's throw some more data into our pipeline.  We'll draw 10 random rows from our spreadsheet, perform an inference from that, and then display the results and the logs showing the pipeline's actions.

In [14]:
ix = numpy.random.choice(purchases.shape[0], size=10, replace=False)
converted = conversion.pandas_to_dict(purchases.iloc[ix,: ])
test_df = pd.DataFrame(converted['query'], columns=converted['colnames'])
display(test_df)

output = demandcurve_pipeline.infer(test_df)
display(output)

,Date,cust_known,StockCode,UnitPrice,UnitsSold
0,2011-09-09,False,20712,2.08,4
1,2010-12-07,True,21928,1.95,21
2,2011-10-11,True,23203,2.08,40
3,2011-01-26,False,22385,4.13,1
4,2011-09-04,True,23202,2.08,26
5,2011-02-20,True,21929,1.95,10
6,2010-12-09,False,22663,5.06,1
7,2010-12-01,False,85099F,4.21,1
8,2011-07-25,False,22411,4.13,3
9,2011-11-11,True,21928,1.79,100


,time,in.Date,in.StockCode,in.UnitPrice,in.UnitsSold,in.cust_known,out.prediction,anomaly.count
0,2024-04-16 19:41:56.287,2011-09-09,20712,2.08,4,False,[9.110871233285868],0
1,2024-04-16 19:41:56.287,2010-12-07,21928,1.95,21,True,[40.57067616108544],0
2,2024-04-16 19:41:56.287,2011-10-11,23203,2.08,40,True,[33.125327529877765],0
3,2024-04-16 19:41:56.287,2011-01-26,22385,4.13,1,False,[6.7715457962084376],0
4,2024-04-16 19:41:56.287,2011-09-04,23202,2.08,26,True,[33.125327529877765],0
5,2024-04-16 19:41:56.287,2011-02-20,21929,1.95,10,True,[40.57067616108544],0
6,2024-04-16 19:41:56.287,2010-12-09,22663,5.06,1,False,[5.71029113645628],0
7,2024-04-16 19:41:56.287,2010-12-01,85099F,4.21,1,False,[6.680255142999893],0
8,2024-04-16 19:41:56.287,2011-07-25,22411,4.13,3,False,[6.7715457962084376],0
9,2024-04-16 19:41:56.287,2011-11-11,21928,1.79,100,True,[49.73419582342663],0


## Undeploy the Pipeline

Once we've finished with our demand curve demo, we'll undeploy the pipeline and give the resources back to our Kubernetes cluster.

In [15]:
demandcurve_pipeline.undeploy()

name,demandcurvepipeline
created,2024-04-16 19:38:36.714333+00:00
last_updated,2024-04-16 19:40:58.596389+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"182cdcee-ef04-416e-a4dd-964e3b3a2e9c, 79b65e34-675d-4bdc-b764-1299f30fa26f"
steps,demandcurvemodel
published,False


Thank you for being a part of this demonstration.  If you have additional questions, please feel free to contact us at Wallaroo.